# Simplex Algorithm

## Introduction

> “The simplex method is a method for solving problems in linear programming. This method, invented by George Dantzig in 1947, tests adjacent vertices of the feasible set (which is a polytope) in sequence so that at each new vertex the objective function improves or is unchanged.” (Wolfram MathWorld)

The underlying observation is that our linear program bears some resemblance with a linear equation system:

$\mathbf{x} \leq \mathbf{b} \leftrightarrow \mathbf{x} +  \mathbf{s} =  \mathbf{b} \text{ with }  \mathbf{s} \geq 0$

If there are n+m variables in a system of m equations (where n>m),
by setting n variables to zero we can use the remaining m variables and solve the equations.

### Basic Solution
For LP problems in normal form with 𝑚 linear inequalities with 𝑛 variables a basic solution is obtained by setting any 𝑛  of the 𝑛+𝑚 variables to zero and solving for the remaining 𝑚 variables
the 𝑚 non-zero variables are called _basic variables_ the remaining 𝑛 zero-valued variables are called _non-basic variables_.

**In our toy factory example:**
Heuristic solution: 40 soldiers, 20 trains, 20 units of carpenting leftover (0 leftover finishing, 0 leftover production allowance for soldiers)

Other variants:
No leftover carpenting, no leftover finishing

No leftover carpenting, no leftover allowance

Produce nothing

produce no soldiers with no carpenting left

produce no soldiers with no finishing left

produce no soldiers with no maxsoldiers left
produce no trains with no finishing left
produce no trains with no carpenting left
produce no trains with no maxsoldiers left …



In [4]:
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [1]:
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

In [ ]:
def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column


In [ ]:
import math

def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column


In [ ]:
import numpy as np

def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau


In [ ]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions


In [ ]:
c = [3,2,0,0,0]
A = [
    [1, 1,1,0,0],
    [ 2, 1,0,1,0],
    [ 1,0,0,0,1]
]
b = [80,100,40]


In [ ]:
solution = simplex(c, A, b)
print('solution: ', solution)